In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

# Get basic players information for all players
base_url = "https://sofifa.com/players?offset="
columns = ['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Overall', 'Potential', 'Club', 'Value', 'Wage', 'Special']
data = pd.DataFrame(columns = columns)
data

,ID,Name,Age,Photo,Nationality,Overall,Potential,Club,Value,Wage,Special


In [2]:
for offset in range(0, 175):
    url = base_url + str(offset * 60)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    table_body = soup.find('tbody')
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        picture = td[0].find('img').get('data-src')
        pid = td[0].find('img').get('id')
        nationality = td[1].find('a').find('img').get('title')
        #flag_img = td[1].find('img').get('data-src')
        name = td[1].findAll('a')[0].text
        age = td[2].text.strip()
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find('a').text
        #club_logo = td[5].find('img').get('data-src')
        value = td[6].text.strip()
        wage = td[7].text.strip()
        special = td[8].text.strip()
        player_data = pd.DataFrame([[pid, name, age, picture, nationality, overall, potential, club, value, wage, special]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
data = data.drop_duplicates()

In [3]:
data

,ID,Name,Age,Photo,Nationality,Overall,Potential,Club,Value,Wage,Special
0,108080,20 S. Lichtsteiner,35,https://cdn.sofifa.com/players/108/080/20_60.png,Switzerland,72,72,FC Augsburg,€875K,€12K,1931
1,232639,R. Doan,22,https://cdn.sofifa.com/players/232/639/21_60.png,Japan,75,83,DSC Arminia Bielefeld,€12.5M,€15K,1850
2,255043,D. Dike,20,https://cdn.sofifa.com/players/255/043/21_60.png,United States,67,80,Barnsley,€2.5M,€2K,1493
3,205452,A. Rüdiger,27,https://cdn.sofifa.com/players/205/452/21_60.png,Germany,81,83,Chelsea,€27.5M,€90K,1866
4,239085,E. Haaland,19,https://cdn.sofifa.com/players/239/085/21_60.png,Norway,87,93,Borussia Dortmund,€122.5M,€72K,2077
...,...,...,...,...,...,...,...,...,...,...,...
9901,256028,I. Achara,22,https://cdn.sofifa.com/players/256/028/21_60.png,Nigeria,59,64,Toronto FC,€350K,€1K,1525
9902,256034,A. Rodriguez,17,https://cdn.sofifa.com/players/256/034/21_60.png,United States,53,73,Colorado Rapids,€230K,€500,839
9903,256036,A. Baldursson,18,https://cdn.sofifa.com/players/256/036/21_60.png,Iceland,58,73,Bologna,€475K,€2K,1564
9904,256047,C. Lungoyi,19,https://cdn.sofifa.com/players/256/047/21_60.png,Switzerland,58,74,FC Lugano,€525K,€1K,1501


In [195]:
# Get detailed player information from player page
detailed_columns = ['Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Contract Valid Until','Release Clause','Club Player Rating','ID']
detailed_data = pd.DataFrame(index = range(0, data.count()[0]), columns = detailed_columns)
detailed_data.ID = data.ID.values
'''
player_data_url = 'https://sofifa.com/player/'
for id in data.ID:
    url = player_data_url + str(id)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    skill_map = {}
    columns = soup.find('div', {'class': 'wrapper'}).find('div', {'class': 'columns'}).findAll('div', {'class': 'column col-4'})
    for column in columns:
        skills = column.findAll('li')
        for skill in skills:
            if(skill.find('label') != None):
                label = skill.find('label').text
                value = skill.text.replace(label, '').strip()
                skill_map[label] = value
    meta_data = soup.find('div', {'class': 'meta'}).text.split(' ')
    length = len(meta_data)
    weight = meta_data[length - 1]
    height = meta_data[length - 2].split('\'')[0] + '\'' + meta_data[length - 2].split('\'')[1].split('\"')[0]
    skill_map["Height"] = height
    skill_map['Weight'] = weight
    print( soup.find('div', {'class': 'card'}).findAll('li'))
    if('Position' in skill_map.keys()):
        if skill_map['Position'] in ('', 'RES', 'SUB'):
            skill_map['Position'] = soup.find('div', {'class': 'meta'}).find('span').text
        if(skill_map['Position'] != 'GK'):
            card_rows =  soup.find('div', {'class': 'card'}).findAll('li')
            for c_row in card_rows:
                attributes = c_row.findAll('div', {'class': re.compile('column col-sm-2 text-center')})
                for attribute in attributes:
                    if(attribute.find('div')):
                        name = ''.join(re.findall('[a-zA-Z]', attribute.text))
                        value = attribute.text.replace(name, '').strip()
                        skill_map[str(name)] = value
    #test =soup.findAll('div', {'class': 'columns right'})
    #print(test[0].findAll('div', {'class': 'column col-4'}))
    sections = soup.findAll('div', {'class': 'columns right'})[0:3]
    #print(sections[1])
    first = sections[0].findAll('div', {'class': 'column col-4'})
    second = sections[1].findAll('div', {'class': 'column col-4'})
    print(second)
    sections = first + second
    for section in sections:
        items = section.find('ul').findAll('li')
        for item in items:
            value = int(re.findall(r'\d+', item.text)[0])
            name = ''.join(re.findall('[a-zA-Z]*', item.text))
            skill_map[str(name)] = value
    for key, value in skill_map.items():
        detailed_data.loc[detailed_data.ID == id, key] = value

full_data = pd.merge(data, detailed_data, how = 'inner', on = 'ID')
full_data.to_csv('data.csv', encoding='utf-8-sig')
'''

'\nplayer_data_url = \'https://sofifa.com/player/\'\nfor id in data.ID:\n    url = player_data_url + str(id)\n    source_code = requests.get(url)\n    plain_text = source_code.text\n    soup = BeautifulSoup(plain_text, \'html.parser\')\n    skill_map = {}\n    columns = soup.find(\'div\', {\'class\': \'wrapper\'}).find(\'div\', {\'class\': \'columns\'}).findAll(\'div\', {\'class\': \'column col-4\'})\n    for column in columns:\n        skills = column.findAll(\'li\')\n        for skill in skills:\n            if(skill.find(\'label\') != None):\n                label = skill.find(\'label\').text\n                value = skill.text.replace(label, \'\').strip()\n                skill_map[label] = value\n    meta_data = soup.find(\'div\', {\'class\': \'meta\'}).text.split(\' \')\n    length = len(meta_data)\n    weight = meta_data[length - 1]\n    height = meta_data[length - 2].split(\'\'\')[0] + \'\'\' + meta_data[length - 2].split(\'\'\')[1].split(\'"\')[0]\n    skill_map["Height"] = he

In [196]:
detailed_data

,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Release Clause,Club Player Rating,ID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108080
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232639
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255043
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205452
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256028
9193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256034
9194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256036
9195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256047


In [197]:
player_data_url = 'https://sofifa.com/player/'
test = ['108080']
for player_id in test: #data.ID
    url = player_data_url + str(player_id)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    profile_card =  soup.find('div', {'class': 'wrapper'}).find('div', {'class': 'columns'}).findAll('div', {'class': 'column col-3'})
    #test123 =soup.find('div', {'class': 'wrapper'}).find('div', {'class': 'columns'}).findAll('div', {'class': 'column col-4'})
    for link1 in profile_card:
        profile = link1.findAll('li',{'class':'bp3-text-overflow-ellipsis'})
        if len(profile) != 0:
            for i in range(0,len(profile)):
                if profile[i].find('label').text == 'Preferred Foot':
                    pref_foot = profile[i].text.strip().replace('Preferred Foot', '')
                elif profile[i].find('label').text == 'Weak Foot':
                     weak_foot = profile[i].text.split()[0]
                elif profile[i].find('label').text == 'Skill Moves':
                    skill_moves = profile[i].text.split()[0]
                elif profile[i].find('label').text == 'International Reputation':
                    inter_repo = profile[i].text.split()[0]
                elif profile[i].find('label').text == 'Work Rate':
                    work_rate = profile[i].text.strip().replace('Work Rate', '')
                elif profile[i].find('label').text == 'Body Type':
                    body_type = profile[i].text.strip().replace('Body Type', '')
                elif profile[i].find('label').text == 'Real Face':
                    real_face = profile[i].find('span').text
                elif profile[i].find('label').text == 'Release Clause':
                    release_cl = profile[i].find('span').text
        team_det = link1.findAll('ul',{'class':'bp3-text-overflow-ellipsis pl text-right'})
        if len(team_det) != 0:
            team_det_li = team_det[0].findAll('li')
            if(len(team_det_li) == 5):
                for j in range(0,len(team_det_li)):
                    if team_det_li[j].find('label') == None:
                        club_player_rating = team_det_li[j].find('span').text
                    elif team_det_li[j].find('label').text == 'Position':
                        play_position = team_det_li[j].text.strip().replace('Position','')
                    elif team_det_li[j].find('label').text == 'Jersey Number':
                        jersey_number = team_det_li[j].text.strip().replace('Jersey Number','')
                    elif team_det_li[j].find('label').text == 'Joined':
                        joined_date = team_det_li[j].text.strip().replace('Joined','')
                    elif team_det_li[j].find('label').text == 'Contract Valid Until':
                        contract_exp_dt = team_det_li[j].text.strip().replace('Contract Valid Until','')
    detailed_data.loc[detailed_data.ID == player_id, 'Preferred Foot'] = pref_foot
    detailed_data.loc[detailed_data.ID == player_id, 'International Reputation'] = inter_repo
    detailed_data.loc[detailed_data.ID == player_id, 'Weak Foot'] = weak_foot
    detailed_data.loc[detailed_data.ID == player_id, 'Skill Moves'] = skill_moves
    detailed_data.loc[detailed_data.ID == player_id, 'Work Rate'] = work_rate
    detailed_data.loc[detailed_data.ID == player_id, 'Body Type'] = body_type
    detailed_data.loc[detailed_data.ID == player_id, 'Real Face'] = real_face
    detailed_data.loc[detailed_data.ID == player_id, 'Position'] = play_position
    detailed_data.loc[detailed_data.ID == player_id, 'Jersey Number'] = jersey_number
    detailed_data.loc[detailed_data.ID == player_id, 'Joined'] = joined_date
    detailed_data.loc[detailed_data.ID == player_id, 'Contract Valid Until'] = contract_exp_dt
    detailed_data.loc[detailed_data.ID == player_id, 'Release Clause'] = release_cl
    detailed_data.loc[detailed_data.ID == player_id, 'Club Player Rating'] = club_player_rating

In [198]:
detailed_data

,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Contract Valid Until,Release Clause,Club Player Rating,ID
0,Right,3,3,3,High/ High,Normal (170-185),Yes,RB,2,"Aug 19, 2019",2020,€1.5M,75,108080
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232639
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,255043
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,205452
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,239085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9192,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256028
9193,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256034
9194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256036
9195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256047


In [199]:
full_data = pd.merge(data, detailed_data, how = 'inner', on = 'ID')
full_data.to_csv('fifa.csv', encoding='utf-8-sig')

In [ ]:
['Preferred Foot', 'International Reputation', 'Weak Foot', 'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position', 'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing', 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes', 'ID']